# Thresholding and Events: Converting Continuous Calcium Traces into Discrete Event Times

In this module, we will learn how to identify neural events from continuous ΔF/F₀ calcium traces. Calcium imaging data is inherently noisy and indirect, so interpreting neural activity requires identifying discrete moments — “events” — that indicate when neurons were likely active. This transition from continuous to event-based data is a critical step in calcium analysis and underpins much of what we do next: aligning events with stimuli, comparing across trials, or computing firing rates. We will explore both simple and advanced ways to extract event times, using tools ranging from NumPy to the OASIS spike inference algorithm.


## Section 1: Detecting Suprathreshold Activity with NumPy

In this section, we will use simple NumPy-based thresholding to identify moments of elevated activity in a calcium trace. The goal is to mark time points where the signal is likely meaningful — that is, when a neuron may have fired. This approach is based on statistical thresholds derived from the trace itself, and while it does not provide precise spike inference, it introduces the idea of event detection and is a helpful stepping stone to more advanced methods. You will use basic descriptive statistics and logical indexing to isolate high-activity points in the signal.

#### Learning Goals:
- Compute baseline and noise statistics using the mean and standard deviation of the ΔF/F₀ trace.
- Set a detection threshold and identify all time points where the signal exceeds this threshold.
- Use `np.where()` and logical masks to extract suprathreshold indices.
- Visualize the calcium trace alongside the detected activity points for verification.


## Section 2: Converting Suprathreshold Frames into Timestamps

Once we have identified where the calcium trace exceeds a threshold, the next step is to determine when those events begin. In this section, we will learn how to detect the **onsets** of suprathreshold activity and convert those indices into timestamps. This helps eliminate redundancy from sustained signals and results in a clean list of candidate event times. You will use `np.diff()` and logical operations to find transitions in the thresholded signal and convert array indices into real-world time using the frame rate.

#### Learning Goals:
- Convert the thresholded trace into a binary (0/1) activity vector.
- Use `np.diff()` to detect transitions from 0 to 1, indicating event onset.
- Map the resulting indices to timestamps using the known sampling interval (`dt`).
- Apply a refractory period if needed to avoid detecting multiple onsets for the same event.

## Section 3: Inferring Spike Events with the OASIS Algorithm

In this section, we will introduce a more advanced method for extracting spike events from calcium traces using the OASIS algorithm. OASIS fits a generative model of calcium dynamics to estimate the most likely times of neuronal firing, returning both spike amplitudes and deconvolved activity. This approach improves temporal resolution and accounts for the shape of calcium transients, making it particularly suitable for precise event detection in high-throughput datasets. You will learn how to apply the `deconvolve()` function from the `oasis` package and extract spike timings from its output.

#### Learning Goals:
- Install and import the OASIS package and load a calcium trace for analysis.
- Use the `deconvolve()` function to obtain spike train (`s`), denoised trace (`c`), and baseline (`b`).
- Extract non-zero entries from the spike train and convert them to timestamps and amplitudes.
- Apply a minimum amplitude threshold to remove low-confidence events if needed.

## Section 4: Visualizing Firing Rate from Event Timestamps

With a list of inferred spike times, we can now compute and visualize firing rates — a central measure in neuroscience data analysis. In this section, we will convert event timestamps into a time-resolved firing rate trace using binning methods. You will learn how to create histograms of spike counts over time, normalize the data to obtain rates in hertz, and apply optional smoothing for improved interpretability. Firing rate traces allow for direct comparisons across neurons, conditions, or experimental time windows.

#### Learning Goals:
- Define time bins using `np.arange()` based on the trace duration and desired temporal resolution.
- Count the number of events in each bin using `np.histogram()`.
- Normalize spike counts to obtain firing rates in Hz (`rate = count / bin_width`).
- Plot the resulting firing rate trace and apply smoothing using convolution or filtering functions.